# Linear Regression with PySyft

## General imports

In [1]:
import numpy as np
import torch

## Import Pysyft and setting virtual workers

In [2]:
import syft as sy  # import the Pysyft library

# hook PyTorch to add extra functionalities like Federated and Encrypted Learning
hook = sy.TorchHook(torch) 

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
james = sy.VirtualWorker(hook, id="james")
jon = sy.VirtualWorker(hook, id="james").add_worker(sy.local_worker)

workers = [bob, alice, james]
crypto_provider = jon

W0806 14:31:40.023433 4716291520 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/Users/andre.farias/pysyft-env/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0806 14:31:40.080029 4716291520 deprecation_wrapper.py:119] From /Users/andre.farias/pysyft-env/lib/python3.7/site-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0806 14:31:41.170000 4716291520 base.py:646] Worker me already exists. Replacing old worker which could cause                     unexpected behavior


## Creating simulated data

As the objective of this tutorial is to simulated data for a Linear Regression, I will generate data folowing the linear model assumption:

$$ y = \beta  X + \epsilon $$

Where $ X $ is a matrix that corresponds to the independent input variables or features, $ y $ corresponds to the output variable that depends on the inputs, $ \beta $ are the coefficients we are trying to model and $ \epsilon $ is  the noise that we suppose to be drawn from a normal distribution with mean 0.

For this simulated data I will suppose:
- There are 300 samples distributed equally across 3 workers
- There are 10 independent variable (or features)
- The "real" coefficients are $\beta = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]$

In [3]:
N = 300 # number of samples
K = 10 # number of features

X = torch.rand((N, K))*10 # simulated input
beta = torch.Tensor(np.linspace(1, 10, 10)).view(-1,1) # "real" coefficients
eps = torch.randn((N,1)) # simulated noise

y = torch.matmul(X, beta) + eps # simulated output

Now, the objective is to simulated distributed data on 3 workers and solve the linear regression problem to obtain the best estimator $\hat{\beta}$.

For this I will first send a different chunk of data for each worker:

In [5]:
chunk_size = int(N/len(workers))
X_chunks = []
y_chunks = []
sizes = []
for i, worker in enumerate(workers):
    X_chunks.append(X[i*chunk_size:(i+1)*chunk_size].send(worker))
    y_chunks.append(y[i*chunk_size:(i+1)*chunk_size].send(worker))
    sizes.append(torch.Tensor([chunk_size]).send(worker))
    
# Keeping the pointers to each chunk in a MultiPointerTensor
X_mpt = sy.MultiPointerTensor(children=X_chunks)
y_mpt = sy.MultiPointerTensor(children=y_chunks)
sizes_mpt = sy.MultiPointerTensor(children=sizes)

## Solving Linear Regression with distributed data

Now that we have the simulated environment set up with the data virtually distributed among three different servers, let's train our linear model in a distributed manner.

### I. Plaintext version

The first and simpler way to solve the Linear Regression problem with distributed data does not need encrypted computation. The main idea is to separate the solution in two stages:

- **Compress**: compute the following aggregates separately in each server:

$$ y^T y, \;\; X^T y, \;\; X^T X $$

- **Combine**: cumpute the matrices aggregates and $N$ for the whole model by summing the aggregates between workers and the sample size of each worker, and then  obtain $\hat{\beta}$ and its standard error from it

When combining the aggregates without encrypted computation we leak some information about the data. However it is important to note that when we have large amounts of data, thanks to the **compress** stage, it's very difficult to revert these aggregates to the original data. 

Moreover, thanks to this two-stage process we are able to perform a Linear Regression in a distributed manner while speeding the computation up: for big $N$ and small $K$ we reduce the computation time by performing **compression** in parallel (which has complexity $\mathcal{O}(NK^2 / n_{workers})$ ) and then performing **combination** (which  has complexity $\mathcal{O}(K^3)$)

#### Compress stage: computing aggregates

In [6]:
# Computing matrices aggregates multipointers
yTy_mpt = y_mpt.transpose(0,1).mm(y_mpt)
XTy_mpt = X_mpt.transpose(0,1).mm(y_mpt)
XTX_mpt = X_mpt.transpose(0,1).mm(X_mpt)

#### Combine stage: summing aggregates and solving Linear Regression

In [7]:
# Sending aggregates to local worker using the method .get() and 
# summing them to obtain the whole aggregates
yTy = sum(yTy_mpt.get())
XTy = sum(XTy_mpt.get())
XTX = sum(XTX_mpt.get())
N = sum(sizes_mpt.get())

In order to obtain the mean $\hat{\beta}$ we need to solve the following linear system:

$$ (X^TX)\hat{\beta} = X^T y $$

The variance of $\beta$ is given by:

$$ (X^TX)^{-1} \sigma^2 $$

Where sigma is obtained from its unbiased estimator:

$$ \hat{\sigma}^2 = \frac{y^Ty - \hat{\beta}^T (X^TX)\hat{\beta}}{N - K} $$ 

For this version of the solution (plaintext with no encrypted computation) we will only need a library to compute linear algebra operations such as linear system solving and matrix inversion:

In [8]:
# Solving with Linear Algebra
yTy = np.array(yTy)
XTy = np.array(XTy)
XTX = np.array(XTX)
N = np.array(N)

beta_hat = np.linalg.solve(XTX, XTy)

sig2_hat = (yTy - beta_hat.T @ XTX @ beta_hat) / (N - K)

var = np.linalg.inv(XTX) * sig2_hat

beta_std = np.sqrt(np.diag(var))

Now let's display our results and compare with the "real" values:

In [9]:
def display_results(coeffs, std_errors, sigma2):
    print("Coefficients | Standard error")
    print("-----------------------------")
    for coef, std in zip(coeffs.squeeze(), std_errors):
        print("    {:.3f} ".format(coef),"  | ", "   {:.3f}".format(std))
    print("-----------------------------")
    print("   Noise variance: {:.3f}".format(sigma2.squeeze()))
    print("-----------------------------")
    
display_results(beta_hat, beta_std, sig2_hat)

Coefficients | Standard error
-----------------------------
    0.986    |     0.020
    2.017    |     0.019
    3.038    |     0.020
    3.990    |     0.019
    4.998    |     0.019
    5.990    |     0.021
    6.976    |     0.019
    7.991    |     0.020
    9.016    |     0.020
    9.997    |     0.019
-----------------------------
   Noise variance: 1.034
-----------------------------


We can notice our computed coefficients are close to the "real" ones, and the variance of noise is close to 1 as expected.